In [52]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score,mean_absolute_error , mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder


In [53]:
df=pd.read_csv(r'data/cement_data_cleaned.csv')

In [54]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [55]:
X.shape,y.shape

((1005, 8), (1005,))

In [56]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)

In [57]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 8 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Cement (component 1)(kg in a m^3 mixture)              1005 non-null   float64
 1   Blast Furnace Slag (component 2)(kg in a m^3 mixture)  1005 non-null   float64
 2   Fly Ash (component 3)(kg in a m^3 mixture)             1005 non-null   float64
 3   Water  (component 4)(kg in a m^3 mixture)              1005 non-null   float64
 4   Superplasticizer (component 5)(kg in a m^3 mixture)    1005 non-null   float64
 5   Coarse Aggregate  (component 6)(kg in a m^3 mixture)   1005 non-null   float64
 6   Fine Aggregate (component 7)(kg in a m^3 mixture)      1005 non-null   float64
 7   Age (day)                                              1005 non-null   int64  
dtypes: float64(7), int64(1)
memory usage: 62.9 KB


In [58]:
df.columns

Index(['Cement (component 1)(kg in a m^3 mixture)',
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)',
       'Fly Ash (component 3)(kg in a m^3 mixture)',
       'Water  (component 4)(kg in a m^3 mixture)',
       'Superplasticizer (component 5)(kg in a m^3 mixture)',
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)',
       'Fine Aggregate (component 7)(kg in a m^3 mixture)', 'Age (day)',
       'Concrete compressive strength(MPa, megapascals) '],
      dtype='object')

In [59]:
target_column = df.columns[-1]
print(target_column)


Concrete compressive strength(MPa, megapascals) 


Here all feature is numerical so only numerical pipeline create here


In [60]:
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object', 'category']).columns

In [61]:
num_col=X.select_dtypes(exclude='object').columns
num_col.to_list

<bound method IndexOpsMixin.tolist of Index(['Cement (component 1)(kg in a m^3 mixture)',
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)',
       'Fly Ash (component 3)(kg in a m^3 mixture)',
       'Water  (component 4)(kg in a m^3 mixture)',
       'Superplasticizer (component 5)(kg in a m^3 mixture)',
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)',
       'Fine Aggregate (component 7)(kg in a m^3 mixture)', 'Age (day)'],
      dtype='object')>

In [62]:
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [63]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols)
])


In [64]:
num_tree_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

cat_tree_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [65]:
tree_preprocessor = ColumnTransformer([
    ('num', num_tree_pipeline, num_cols),
    ('cat', cat_tree_pipeline, cat_cols)
])

In [66]:
import warnings
warnings.filterwarnings('ignore')

X_train_scaled = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())

X_test_scaled = pd.DataFrame(preprocessor.transform(X_test), columns=preprocessor.get_feature_names_out())

In [68]:
X_train_scaled.head()

,num_pipeline__Cement (component 1)(kg in a m^3 mixture),num_pipeline__Blast Furnace Slag (component 2)(kg in a m^3 mixture),num_pipeline__Fly Ash (component 3)(kg in a m^3 mixture),num_pipeline__Water (component 4)(kg in a m^3 mixture),num_pipeline__Superplasticizer (component 5)(kg in a m^3 mixture),num_pipeline__Coarse Aggregate (component 6)(kg in a m^3 mixture),num_pipeline__Fine Aggregate (component 7)(kg in a m^3 mixture),num_pipeline__Age (day)
0,-1.078725,2.136608,-0.861406,0.021226,1.154937,-1.446165,0.696974,-0.277746
1,-0.848293,-0.855616,1.097172,-0.957018,0.635153,1.480293,0.396097,-0.277746
2,-1.220733,-0.855616,2.033122,-0.718816,1.654731,1.300184,-1.139592,-0.277746
3,-0.578383,-0.855616,1.080744,-1.112426,0.876719,1.360220,0.301434,-0.671365
4,0.502300,-0.855616,-0.861406,0.437500,-1.010833,-0.523903,0.889258,0.698429


In [70]:
models={
                "Linear Regression":LinearRegression(),
                "Lasso":Lasso(),
                "Ridge":Ridge(),
                "Decision Tree":DecisionTreeRegressor(),
                "Random Forest Regressor":RandomForestRegressor(),
                "Adaboost Regressor":AdaBoostRegressor(),
                "GradiantBoost Regressor":GradientBoostingRegressor(),
                "KNN Regressor":KNeighborsRegressor(),
                'SVR': SVR()
            }


In [71]:
def evalute_models(true,predicted):
    r2 = r2_score(true, predicted)
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    return r2, mae, rmse

In [72]:
model_list=[]
r2_list=[]
def modelEvalute(X_train,y_train,X_test,y_test,models):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]

        model.fit(X_train_scaled,y_train)

        y_pred_non_tree=model.predict(X_test_scaled)
        score,mae,rmse=evalute_models(y_test,y_pred_non_tree)

        report[list(models.keys())[i]]=score

        

        
    return report



In [73]:
model_report=modelEvalute(X_train_scaled,y_train,X_test_scaled,y_test,models)
model_report

{'Linear Regression': 0.6422265413538712,
 'Lasso': 0.5718108268835684,
 'Ridge': 0.6422012011683127,
 'Decision Tree': 0.7982186939334577,
 'Random Forest Regressor': 0.8962866849923673,
 'Adaboost Regressor': 0.7696361027043707,
 'GradiantBoost Regressor': 0.8846253618465405,
 'KNN Regressor': 0.7191485110111079,
 'SVR': 0.6569372021332041}

In [74]:
best_model_score=max(sorted(model_report.values()))

In [75]:
best_model_score


0.8962866849923673

## here we can see that Random forest regressor is best performance so we can use that for model
### and define parameters for hyper parameter tuning



In [76]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

rf = RandomForestRegressor(random_state=42)

grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=2
)

grid.fit(X_train_scaled, y_train)



Fitting 5 folds for each of 48 candidates, totalling 240 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",RandomForestR...ndom_state=42)
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'bootstrap': [True, False], 'max_depth': [None, 10, ...], 'max_features': ['sqrt', 'log2'], 'n_estimators': [100, 200, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'neg_mean_squared_error'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for e

In [77]:
grid.best_params_

{'bootstrap': False,
 'max_depth': 20,
 'max_features': 'log2',
 'n_estimators': 300}

In [78]:
y_pred=grid.predict(X_test_scaled)

In [79]:
models={
                "Linear Regression":LinearRegression(),
                "Lasso":Lasso(),
                "Ridge":Ridge(),
                "Decision Tree":DecisionTreeRegressor(),
                "Random Forest Regressor":RandomForestRegressor(),
                "Adaboost Regressor":AdaBoostRegressor(),
                "GradiantBoost Regressor":GradientBoostingRegressor(),
                "KNN Regressor":KNeighborsRegressor(),
                'SVR': SVR()
            }

In [80]:
param = {

                "Linear Regression": {
                #"fit_intercept": [True, False]
                },

                "Lasso": {
                    "alpha": [0.001, 0.01, 0.1, 1, 10],
                    #"max_iter": [1000, 5000]
                },

                "Ridge": {
                    "alpha": [0.1, 1, 10, 50],
                    "solver": ["auto", "svd", "cholesky"]
                },

                "Decision Tree": {
                    "criterion": ["squared_error", "friedman_mse"],
                    "max_depth": [None, 5,3, 10, 20],
                   # "min_samples_split": [2, 5, 10],
                    #"min_samples_leaf": [1, 2, 4]
                },

                "Random Forest Regressor": {
                    "n_estimators": [100, 200, 300],
                    "max_depth": [None, 10, 20],
                    #"min_samples_split": [2, 5],
                    #"min_samples_leaf": [1, 2],
                    "max_features": ["sqrt", "log2"]
                },

                "Adaboost Regressor": {
                    "n_estimators": [50, 100, 200],
                    "learning_rate": [0.01, 0.1, 1]
                },

                "GradiantBoost Regressor": {
                    "n_estimators": [100, 200],
                    "learning_rate": [0.01, 0.1],
                    "max_depth": [3, 5],
                    #"subsample": [0.8, 1.0]
                },

                "KNN Regressor": {
                    "n_neighbors": [3, 5, 7, 9],
                    "weights": ["uniform", "distance"],
                    #"metric": ["euclidean", "manhattan"]
                },

                "SVR" : {
                    "kernel": ["rbf", "linear", "poly"],
                    "C": [0.1, 1, 10, 100],
                    # "epsilon": [0.01, 0.1, 0.2],
                    # "gamma": ["scale", "auto"]
                }
            }

In [81]:

def evaluate_model(X_train,y_train,X_test,y_test,models,param):
        report={}
        for i in range(len(models)):
            model=list(models.values())[i]
            para=param[list(models.keys())[i]]
            #train model
            gs=GridSearchCV(model,para,cv=3)
            gs.fit(X_train,y_train)

            model.set_params(**gs.best_params_)

            model.fit(X_train,y_train)

            y_train_pred=model.predict(X_train)
            y_test_pred=model.predict(X_test)

            #train_model_score=r2_score(X_train,y_train)
            test_model_score=r2_score(y_test,y_test_pred)

            report[list(models.keys())[i]] = test_model_score
        
        return report

In [83]:
model_report:dict=evaluate_model(X_train,y_train,X_test,y_test,models,param)
 

In [ ]:
best_model_score=max(sorted(model_report.values()))
best_model_name=list(model_report.keys())[list(model_report.values()).index(best_model_score)]
best_model=models[best_model_name]

print(f"best model is :{best_model_name},R2 score : {best_model_score}")
print("="*12)

best model is :Random Forest Regressor,R2 score : 0.9117989704945796
